In [3]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Create output directory
output_dir = Path('visualizations')
output_dir.mkdir(exist_ok=True)

In [4]:
# Cell 2: Load Your Ensemble Results
# Update this path to your actual JSON file
results_path = '../results/result.json'  # Change to your file path

with open(results_path, 'r') as f:
    results = json.load(f)

print("Loaded ensemble combinations:")
for combo in results.keys():
    if '+' in combo:  # Only show combinations
        print(f"- {combo}")

Loaded ensemble combinations:


In [5]:
combinations = {}
individual_models = set()

for model_name, model_data in results.items():
    if '+' in model_name:
        # This is a combination
        parts = model_name.split(' + ')
        combinations[model_name] = {
            'models': parts,
            'metrics': model_data['metrics'],
            'weights': model_data.get('optimal_weight', [0.5, 0.5])
        }
        individual_models.update(parts)
    else:
        # This is an individual model
        individual_models.add(model_name)

print(f"\nIndividual models: {individual_models}")
print(f"Number of combinations: {len(combinations)}")


Individual models: {'diversity_metrics', 'retrieval_results', 'experiment_info', 'emotion_analysis', 'ranking_results'}
Number of combinations: 0


In [6]:
k_value = '10'
metric = 'hit_rate'

# Create a matrix for the heatmap
models = sorted(list(individual_models))
n_models = len(models)
performance_matrix = np.zeros((n_models, n_models))
weight_matrix = np.zeros((n_models, n_models, 2))  # Store weights

# Fill the matrix
for i, model1 in enumerate(models):
    for j, model2 in enumerate(models):
        if i == j:
            # Diagonal: individual model performance
            if model1 in results:
                performance_matrix[i, j] = results[model1]['metrics'][metric][k_value]
        else:
            # Off-diagonal: combined performance
            combo_name1 = f"{model1} + {model2}"
            combo_name2 = f"{model2} + {model1}"
            
            if combo_name1 in combinations:
                performance_matrix[i, j] = combinations[combo_name1]['metrics'][metric][k_value]
                weight_matrix[i, j] = combinations[combo_name1]['weights']
            elif combo_name2 in combinations:
                performance_matrix[i, j] = combinations[combo_name2]['metrics'][metric][k_value]
                # Reverse weights for reversed combination
                weight_matrix[i, j] = combinations[combo_name2]['weights'][::-1]

# Create the heatmap
plt.figure(figsize=(12, 10))
mask = np.zeros_like(performance_matrix)
mask[np.triu_indices_from(mask, k=1)] = True  # Mask upper triangle

sns.heatmap(performance_matrix, 
            annot=True, 
            fmt='.1f',
            cmap='YlOrRd',
            xticklabels=models,
            yticklabels=models,
            mask=mask,
            square=True,
            linewidths=0.5,
            cbar_kws={'label': f'{metric.upper()}@{k_value} (%)'},
            vmin=0,
            vmax=performance_matrix.max())

plt.title(f'Ensemble Performance Heatmap - {metric.upper()}@{k_value}', fontsize=16, fontweight='bold')
plt.xlabel('Model 2', fontsize=12)
plt.ylabel('Model 1', fontsize=12)

# Add text annotation for interpretation
plt.text(0.5, -0.1, 'Diagonal: Individual model performance\nLower triangle: Ensemble performance', 
         transform=plt.gca().transAxes, ha='center', fontsize=10, style='italic')

plt.tight_layout()
plt.savefig(output_dir / f'ensemble_heatmap_{metric}_k{k_value}.png', dpi=300, bbox_inches='tight')
plt.show()

KeyError: 'metrics'